[快速开始-使用文档-PaddlePaddle深度学习平台](https://www.paddlepaddle.org.cn/documentation/docs/zh/1.5/beginners_guide/quick_start_cn.html)
计算9a+5b+2c+10d的值

In [12]:
import paddle
paddle.enable_static()
#加载库
import paddle.fluid as fluid
import numpy as np

In [14]:
#生成数据
np.random.seed(0)
outputs = np.random.randint(5, size=(10, 4))
res = []
for i in range(10):
  # 假设方程式为 y=4a+6b+7c+2d
  y = 4*outputs[i][0]+6*outputs[i][1]+7*outputs[i][2]+2*outputs[i][3]
  res.append([y])

In [15]:
# 定义数据
train_data=np.array(outputs).astype('float32')
y_true = np.array(res).astype('float32')

In [21]:
#定义网络
x = fluid.layers.data(name="x",shape=[4],dtype='float32')
y = fluid.layers.data(name="y",shape=[1],dtype='float32')
y_predict = fluid.layers.fc(input=x,size=1,act=None)
#定义损失函数
cost = fluid.layers.square_error_cost(input=y_predict,label=y)
avg_cost = fluid.layers.mean(cost)
#定义优化方法
sgd_optimizer = fluid.optimizer.SGD(learning_rate=0.05)
sgd_optimizer.minimize(avg_cost)
#参数初始化
cpu = fluid.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program())

[]

In [26]:
##开始训练，迭代500次
for i in range(500):
  outs = exe.run(feed={'x':train_data,'y':y_true},fetch_list=[y_predict.name,avg_cost.name])
  if i%50==0:print ('iter={:.0f},cost={}'.format(i,outs[1][0]))

iter=0,cost=1.5329645975725725e-05
iter=50,cost=6.3514394241792616e-06
iter=100,cost=2.6318452910345513e-06
iter=150,cost=1.0899177596002119e-06
iter=200,cost=4.5169218765295227e-07
iter=250,cost=1.8701830128975416e-07
iter=300,cost=7.792332468170571e-08
iter=350,cost=3.1904711761399085e-08
iter=400,cost=1.3530370601699815e-08
iter=450,cost=5.383844925432868e-09


In [23]:
#存储训练结果
params_dirname = "result"
fluid.io.save_inference_model(params_dirname, ['x'], [y_predict], exe)

['save_infer_model/scale_0.tmp_1']

In [27]:

# 开始预测
infer_exe = fluid.Executor(cpu)
inference_scope = fluid.Scope()
# 加载训练好的模型
with fluid.scope_guard(inference_scope):
    [inference_program, feed_target_names,fetch_targets] = fluid.io.load_inference_model(params_dirname, infer_exe)

# 生成测试数据
test = np.array([[[9],[5],[2],[10]]]).astype('float32')
# 进行预测
results = infer_exe.run(inference_program,
                        feed={"x": test},
                        fetch_list=fetch_targets)
# 给出题目为 【9,5,2,10】 输出y=4*9+6*5+7*2+10*2的值
print ("9a+5b+2c+10d={}".format(results[0][0]))

9a+5b+2c+10d=[99.99961]
